In [3]:
import pygame
import random
import math

# Inicializar o Pygame
pygame.init()

# Configurações da Janela
WINDOW_SIZE = 400
GRID_SIZE = 100  # Mapa de 100x100, escalado para caber na janela
CELL_SIZE = WINDOW_SIZE // GRID_SIZE
SCREEN = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
pygame.display.set_caption("Simulação de Agentes")

# Cores
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Classe do Agente
class Agent:
    def __init__(self, x, y, hunger, vision):
        self.x = x
        self.y = y
        self.hunger = hunger
        self.vision = vision
        self.alive = True

    def move(self):
        """Move aleatoriamente dentro do mapa"""
        if not self.alive:
            return
        dx, dy = random.choice([(0, 1), (1, 0), (0, -1), (-1, 0)])
        self.x = max(0, min(self.x + dx, GRID_SIZE - 1))
        self.y = max(0, min(self.y + dy, GRID_SIZE - 1))
        self.hunger += 1  # Fome aumenta a cada movimento

    def move_towards(self, target_x, target_y):
        """Move o agente em direção à comida"""
        if not self.alive:
            return
        if self.x < target_x:
            self.x += 1
        elif self.x > target_x:
            self.x -= 1
        if self.y < target_y:
            self.y += 1
        elif self.y > target_y:
            self.y -= 1
        self.hunger += 1

    def see_food(self, food_x, food_y):
        """Verifica se a comida está no raio de visão"""
        if not self.alive:
            return False
        distance = math.sqrt((self.x - food_x) ** 2 + (self.y - food_y) ** 2)
        return distance <= self.vision

    def die_if_hungry(self):
        """Mata o agente se ele estiver com fome demais"""
        if self.hunger >= 100:
            self.alive = False


# Configurações da Simulação
NUM_AGENTS = 100
FOOD_LIST = [(random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))]  # Lista de comidas
AGENTS = [Agent(random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1), 0, 10) for _ in range(NUM_AGENTS)]
FOOD_RESPAWN_INTERVAL = 50  # Intervalo em frames para adicionar nova comida

# Loop Principal
clock = pygame.time.Clock()
frame_count = 0
running = True
while running:
    SCREEN.fill(WHITE)
    frame_count += 1

    # Eventos do Pygame
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Adicionar nova comida em intervalos regulares
    if frame_count % FOOD_RESPAWN_INTERVAL == 0:
        FOOD_LIST.append((random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)))

    # Atualização dos agentes
    for agent in AGENTS:
        if not agent.alive:
            continue

        # Verificar se o agente vê alguma comida
        for food in FOOD_LIST:
            if agent.see_food(*food):
                agent.move_towards(*food)
                if (agent.x, agent.y) == food:
                    print(f"Agente {agent} comeu a comida!")
                    agent.hunger = 0
                    FOOD_LIST.remove(food)
                break  # Só persegue uma comida por vez
        else:
            agent.move()

        agent.die_if_hungry()

        # Desenhar o agente
        color = GREEN if agent.alive else RED
        pygame.draw.rect(SCREEN, color, (agent.x * CELL_SIZE, agent.y * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Desenhar as comidas
    for food in FOOD_LIST:
        pygame.draw.rect(SCREEN, BLUE, (food[0] * CELL_SIZE, food[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))

    # Atualizar a tela
    pygame.display.flip()
    clock.tick(10)  # FPS

# Finalizar o Pygame
pygame.quit()


Agente <__main__.Agent object at 0x000001F968D39A90> comeu a comida!
Agente <__main__.Agent object at 0x000001F968D39CA0> comeu a comida!
